In [3]:
import datetime
import os

# --- 1. Define Search Parameters ---

# Set a flexible time range. 
# MODIS/SWOT cover 2023+. PACE data is only available from March 2024 onwards.
START_DATE = "2023-01-01T00:00:00Z" # Keeps the full 3-year window for MODIS/SWOT
END_DATE = "2025-12-31T23:59:59Z"

# !!! IMPORTANT: VERIFY AND ADJUST YOUR AREA OF INTEREST (AOI) !!!
# Example: Western Atlantic/Gulf Stream (-85 W to -50 W, 20 N to 45 N)
BOUNDING_BOX = (-85.0, 20.0, -50.0, 45.0)

DOWNLOAD_DIR = "nasa_shark_data_revised_2"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)


# --- 2. Define REVISED NASA Data Collections (Short Names) ---

DATA_COLLECTIONS = {
    # *** RECOVERY MISSION 1: PACE (Chlorophyll & NPP) ***
    # PACE L3/L4 products start availability in March 2024. 
    # We will search the full time window; the CMR will only return 2024+ files.
    "PACE_CHL": "PACE_OCI_L3M_CHL_V3",  # High-res, new sensor
    "PACE_NPP": "PACE_OCI_L4_NPP_V1",    # Level 4 data product
    
    # *** RECOVERY MISSION 2: MODIS-Aqua (Historical Chlorophyll) ***
    # The short name 'MODISA_L3m_CHL_4km_MONTHLY' is sometimes rejected.
    # We use the generic collection name and constrain to monthly, mapped, and 4km,
    # or a known working name if possible. We will try a different standard L3 name.
    "MODIS_CHL": "MODISA_L3m_CHL", # Generic L3 mapped Chlorophyll short name

    # Data already found: SWOT and SST (Keep these names)
    "SWOT_SSH": "SWOT_L2_LR_SSH_EXPERT_2.0", 
    "SST": "MUR-JPL-L4-GLOB-v4.1" 
}


# --- 3. Run Search and Download for ALL MISSIONS (including successful ones) ---

print("Step 2: Performing Data Search...")

all_results = []
for mission_name, short_name in DATA_COLLECTIONS.items():
    print(f"\n  -> Searching for {mission_name} ({short_name}) from {START_DATE.split('T')[0]} to {END_DATE.split('T')[0]}...")

    # Search the Common Metadata Repository (CMR)
    # We add a constraint for Chlorophyll products to ensure monthly 4km resolution
    if mission_name == "MODIS_CHL":
        # Additional keywords to force monthly 4km selection for the MODIS generic collection
        keywords = ["Monthly", "4km"]
    else:
        keywords = []

    results = earthaccess.search_data(
        short_name=short_name,
        temporal=(START_DATE, END_DATE),
        bounding_box=BOUNDING_BOX,
        keywords=keywords,
        cloud_hosted=True
    )
    
    if results:
        print(f"     ✅ Found {len(results)} granules for {mission_name}.")
        all_results.extend(results)
    else:
        print(f"     ❌ FAILED: Found 0 granules for {mission_name}. Please verify product name and AOI.")


print(f"\nStep 3: Initiating bulk download of {len(all_results)} total files...")
if all_results:
    # This downloads all files to subfolders within the DOWNLOAD_DIR
    earthaccess.download(all_results, DOWNLOAD_DIR)
    print(f"\n✅ Download complete! All data saved to the '{DOWNLOAD_DIR}' folder.")
else:
    print("❌ No data found with the current parameters. Download skipped.")

Step 2: Performing Data Search...

  -> Searching for PACE_CHL (PACE_OCI_L3M_CHL_V3) from 2023-01-01 to 2025-12-31...


ValueError: Unknown key keywords

In [2]:
import earthaccess

# Step 1: Login using your Earthdata credentials

earthaccess.login()

print("hello")

C:\ProgramData\Anaconda3\envs\sharkenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Enter your Earthdata Login username:  devanandhpr
Enter your Earthdata password:  ········


hello


In [2]:
import earthaccess
import os
import time

# --- DEFINE PARAMETERS (Re-use your existing correct values) ---
START_DATE = "2023-01-01T00:00:00Z"
END_DATE = "2025-12-31T23:59:59Z"
BOUNDING_BOX = (-85.0, 20.0, -50.0, 45.0) 
DOWNLOAD_DIR = "nasa_shark_data_recovery_final" # New directory for the final attempt
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# --- RECOVERY DATA COLLECTIONS ---
# Using the specific short names from the last attempt (they were likely correct)
RECOVERY_COLLECTIONS = {
    "PACE_CHL_NRT": "PACE_OCI_L3M_CHL_NRT",
    "PACE_POC_NRT": "PACE_OCI_L3M_POC_NRT", 
    "MODIS_CHL_V2022": "MODISA_L3M_CHL_2022_4KM",
}

# --- Successful Collections (to ensure you redownload these) ---
SUCCESSFUL_COLLECTIONS = {
    "SWOT_SSH": "SWOT_L2_LR_SSH_EXPERT_2.0",
    "SST": "MUR-JPL-L4-GLOB-v4.1"
}

# Combine all collections for a comprehensive search
ALL_COLLECTIONS = {**RECOVERY_COLLECTIONS, **SUCCESSFUL_COLLECTIONS}


# --- 2. Final Comprehensive Search and Download ---

print("Step 2: Performing Comprehensive Data Search (Excluding 'keywords' argument)...")

all_results = []
for mission_name, short_name in ALL_COLLECTIONS.items():
    print(f"\n  -> Searching for {mission_name} ({short_name})...")

    try:
        results = earthaccess.search_data(
            short_name=short_name,
            temporal=(START_DATE, END_DATE),
            bounding_box=BOUNDING_BOX,
            # *** KEY FIX: REMOVED 'keywords=keywords' ARGUMENT ***
            cloud_hosted=True
        )
    except Exception as e:
        # Catch any new unexpected errors
        print(f"     ⚠️ ERROR during search (Check short name/AOI): {e}")
        results = []
    
    
    if results:
        print(f"     ✅ Found {len(results)} granules for {mission_name}!")
        all_results.extend(results)
    else:
        print(f"     ❌ FAILED: Found 0 granules for {mission_name}. Trying next option.")
    
    time.sleep(1) 


print(f"\nStep 3: Initiating bulk download of {len(all_results)} total files...")
if all_results:
    # This downloads all files, including the re-found SWOT/SST data
    earthaccess.download(all_results, DOWNLOAD_DIR)
    print(f"\n✅ FINAL DOWNLOAD COMPLETE! All available data saved to '{DOWNLOAD_DIR}'.")
else:
    print("❌ No data found with the current parameters. Download skipped.")

C:\ProgramData\Anaconda3\envs\sharkenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Step 2: Performing Comprehensive Data Search (Excluding 'keywords' argument)...

  -> Searching for PACE_CHL_NRT (PACE_OCI_L3M_CHL_NRT)...
     ✅ Found 190 granules for PACE_CHL_NRT!

  -> Searching for PACE_POC_NRT (PACE_OCI_L3M_POC_NRT)...
     ✅ Found 190 granules for PACE_POC_NRT!

  -> Searching for MODIS_CHL_V2022 (MODISA_L3M_CHL_2022_4KM)...
     ❌ FAILED: Found 0 granules for MODIS_CHL_V2022. Trying next option.

  -> Searching for SWOT_SSH (SWOT_L2_LR_SSH_EXPERT_2.0)...
     ✅ Found 2438 granules for SWOT_SSH!

  -> Searching for SST (MUR-JPL-L4-GLOB-v4.1)...
     ✅ Found 1007 granules for SST!


'NoneType' object has no attribute 'get': You must call earthaccess.login() before you can download data



Step 3: Initiating bulk download of 3825 total files...

✅ FINAL DOWNLOAD COMPLETE! All available data saved to 'nasa_shark_data_recovery_final'.


In [4]:
import earthaccess
import os
import time

# --- DEFINE PARAMETERS ---
START_DATE = "2023-01-01T00:00:00Z"
END_DATE = "2025-12-31T23:59:59Z"
BOUNDING_BOX = (-85.0, 20.0, -50.0, 45.0) 

# *** CRITICAL FIX: Use an absolute path on a known, spacious drive ***
DOWNLOAD_DIR = r"D:\NASA_SHARK_PROJECT\nasa_shark_data_final" # <-- MODIFY THIS PATH!
os.makedirs(DOWNLOAD_DIR, exist_ok=True)
# ... rest of the code ...

In [ ]:
import earthaccess
import os
import time

# --- DEFINE PARAMETERS (Re-use your existing correct values) ---
START_DATE = "2023-01-01T00:00:00Z"
END_DATE = "2025-12-31T23:59:59Z"
BOUNDING_BOX = (-85.0, 20.0, -50.0, 45.0) 

# *** CRITICAL FIX: Update the DOWNLOAD_DIR to a safe, spacious drive ***
# EXAMPLE: REPLACE "D:\Your_Big_Drive" with your actual path!
DOWNLOAD_DIR = r"D:\Your_Big_Drive\nasa_shark_data_final" 
os.makedirs(DOWNLOAD_DIR, exist_ok=True)


# --- RECOVERY DATA COLLECTIONS ---
# We use the list that includes your previous successful and fixed-name collections.
ALL_COLLECTIONS = {
    # PACE (Hoping for recent data availability)
    "PACE_CHL_NRT": "PACE_OCI_L3M_CHL_NRT",
    "PACE_POC_NRT": "PACE_OCI_L3M_POC_NRT", 
    
    # MODIS (Using the robust version short name)
    "MODIS_CHL_V2022": "MODISA_L3M_CHL_2022_4KM",

    # Successful Collections (SWOT and SST)
    "SWOT_SSH": "SWOT_L2_LR_SSH_EXPERT_2.0",
    "SST": "MUR-JPL-L4-GLOB-v4.1"
}


# --- 3. Final Comprehensive Search and Download ---

print("Step 2: Performing Comprehensive Data Search (Rerunning all missions)...")

all_results = []
successful_downloads = 0

for mission_name, short_name in ALL_COLLECTIONS.items():
    print(f"\n  -> Searching for {mission_name} ({short_name})...")
    
    # Search without the rejected 'keywords' argument
    try:
        results = earthaccess.search_data(
            short_name=short_name,
            temporal=(START_DATE, END_DATE),
            bounding_box=BOUNDING_BOX,
            cloud_hosted=True
        )
    except Exception as e:
        print(f"     ⚠️ ERROR during search: {e}")
        results = []
    
    if results:
        print(f"     ✅ Found {len(results)} granules for {mission_name}!")
        all_results.extend(results)
    else:
        print(f"     ❌ FAILED: Found 0 granules for {mission_name}.")
    
    time.sleep(1) 


print(f"\nStep 3: Initiating bulk download of {len(all_results)} total files to '{DOWNLOAD_DIR}'...")
if all_results:
    # This will now download to the new location on your large drive
    earthaccess.download(all_results, DOWNLOAD_DIR)
    print(f"\n✅ FINAL DOWNLOAD COMPLETE! All available data saved successfully.")
else:
    print("❌ No data found with the current parameters. Download skipped.")

Step 2: Performing Comprehensive Data Search (Rerunning all missions)...

  -> Searching for PACE_CHL_NRT (PACE_OCI_L3M_CHL_NRT)...
     ⚠️ ERROR during search: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
     ❌ FAILED: Found 0 granules for PACE_CHL_NRT.

  -> Searching for PACE_POC_NRT (PACE_OCI_L3M_POC_NRT)...
     ⚠️ ERROR during search: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
     ❌ FAILED: Found 0 granules for PACE_POC_NRT.

  -> Searching for MODIS_CHL_V2022 (MODISA_L3M_CHL_2022_4KM)...
     ⚠️ ERROR during search: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
     ❌ FAILED: Found 0 granules for MODIS_CHL_V2022.

  -> Searching for SWOT_SSH (SWOT_L2_LR_SSH_EXPERT_2.0)...
